## Import Libraries

In [137]:
from enum import Enum
import re
import pandas
import tkinter as tk
import pandastable as pt
from nltk.tree import *

## define an enumeration that lists token types

In [138]:
class Token_type(Enum):  # listing all tokens type
    setq = 1
    setf = 2
    dotimes = 3
    when = 4
    write = 5
    left_brace = 6
    dot=8
    comment = 9
    EqualOp = 10
    LessThanOp = 11
    GreaterThanOp = 12
    NotEqualOp = 13
    PlusOp = 14
    MinusOp = 15
    MultiplyOp = 16
    DivideOp = 17
    Identifier = 18
    Constant = 19
    Error = 20
    right_brace = 21
    NewLine=23
    Atom=24
    read=25
    greaterThanOrEqual=26
    lessThanOrEqual=27
    print= 28
    min=29
    max = 30
    defun=31
    if_statement=32
    and_op=33
    or_op=34
    not_op=35
    incf=36
    decf=37
    mod=38
    string=39
    true=40
    false=41
    defconstant=42
    rem=43

## class token to hold string and token type

In [139]:
class token:
    def __init__(self, lex, token_type):
        self.lex = lex
        self.token_type = token_type

    def to_dict(self):
        return {
            'Lex': self.lex,
            'token_type': self.token_type
        }

## Reserved word Dictionary

In [140]:
ReservedWords = {
    "setq": Token_type.setq,
    "dotimes":Token_type.dotimes,
    "when":Token_type.when,
    "write":Token_type.write,
    "read":Token_type.read,
    "print":Token_type.print,
    "if":Token_type.if_statement,
    "incf":Token_type.incf,
    "decf":Token_type.decf,
    "t":Token_type.true,
    "nil":Token_type.false,
    "defconstant":Token_type.defconstant,
    "mod":Token_type.mod,
    "rem":Token_type.rem
}

## Operators Dictionary

In [141]:
Operators = {".": Token_type.dot,
             ";": Token_type.comment,
             "=": Token_type.EqualOp,
             "+": Token_type.PlusOp,
             "-": Token_type.MinusOp,
             "*": Token_type.MultiplyOp,
             "/": Token_type.DivideOp,
             "<": Token_type.LessThanOp,
             ">": Token_type.GreaterThanOp,
             "(": Token_type.left_brace,
             ")": Token_type.right_brace,
             "\n":Token_type.NewLine,
             "<=":Token_type.lessThanOrEqual,
             ">=":Token_type.greaterThanOrEqual,
             "and":Token_type.and_op,
             "or":Token_type.or_op,
             "not":Token_type.not_op,
             "mod":Token_type.mod
             } 

## Create Lists for Tokens and Errors

In [142]:
Tokens = [] 
Tokens_comment=[]
errors = []

## find_token that return the list of tokens

In [143]:
def find_token(texts):
    for text in texts:
        text = re.sub(r'\(', r' ( ', text)
        text = re.sub(r'\)', r' ) ', text)
        text = re.sub(r'\;', r' ;', text)
        text = re.sub(r'“', r'"', text)
        text = re.sub(r'”', r'"', text)
        lexems =re.findall(r'"[.*]?"|[^\s]+|[\S"]+', text)

        qoute_flag=False
        x=-1
        #print(lexems)
        for le in lexems:
            if qoute_flag==True:
                if le[len(le)-1]=="\"":
                    qoute_flag=False
                continue

            if le[0] == "\"" and le[len(le)-1]!="\"":
                x=text.find("\"",x+1)
                st = text[x:(text.find("\"",x+1))+1]
                x=text.find("\"",x+1)
                if x!=-1:
                    if st[len(st)-1]=="\"":
                        qoute_flag=True
                        new_token = token(st, Token_type.string)
                        Tokens.append(new_token)
                        Tokens_comment.append(new_token)
                        continue

            if (le.lower() in ReservedWords):
                new_token = token(le, ReservedWords[le.lower()])
                Tokens.append(new_token)
                Tokens_comment.append(new_token)

            elif (re.match(";\w*", le)):
                st=text[text.find(";"):]
                if(text.endswith('\n')):
                    new_token = token(st[:len(st)-1], Token_type.comment)
                    Tokens_comment.append(new_token)
                else: 
                    new_token = token(st, Token_type.comment)
                    Tokens_comment.append(new_token)
                break
            
            elif (le in Operators):
                new_token = token(le, Operators[le])
                Tokens.append(new_token)
                Tokens_comment.append(new_token)

            elif (re.match("^[+|-]?\d+(\.[0-9]+)?$", le)):
                new_token = token(le, Token_type.Constant)
                Tokens.append(new_token)
                Tokens_comment.append(new_token)
                
            elif (re.match("^([a-zA-Z][a-zA-Z0-9]*)$", le)):
                new_token = token(le, Token_type.Identifier)
                Tokens.append(new_token)
                Tokens_comment.append(new_token)

            elif (le[0]=="\"" and le[len(le)-1]=="\""):
                new_token = token(le, Token_type.string)
                Tokens.append(new_token)
                Tokens_comment.append(new_token)

            else:
                new_token = token(le, Token_type.Error)
                Tokens.append(new_token)
                Tokens_comment.append(new_token)

## Check Operators

In [144]:
def check_reserverd_word(j):
    if (Tokens[j].token_type == Token_type.when or Tokens[j].token_type == Token_type.read or
         Tokens[j].token_type == Token_type.setq or Tokens[j].token_type == Token_type.Print
         or Tokens[j].token_type == Token_type.dotimes or Tokens[j].token_type == Token_type.if_statement 
         or Tokens[j].token_type == Token_type.incf or Tokens[j].token_type == Token_type.decf):
        return True
    else:
        return False

In [145]:
def check_arithmetic_operator(j):
    if (Tokens[j].token_type == Token_type.PlusOp or Tokens[j].token_type == Token_type.MinusOp or
         Tokens[j].token_type == Token_type.MultiplyOp or Tokens[j].token_type == Token_type.DivideOp or
           Tokens[j].token_type == Token_type.mod or Tokens[j].token_type == Token_type.rem):
        return True
    else:
        return False

In [146]:
def check_logical_operator(j):
    if (Tokens[j].token_type == Token_type.and_op or Tokens[j].token_type == Token_type.or_op or
         Tokens[j].token_type == Token_type.not_op):
        return True
    else:
        return False

In [147]:
def check_comparison_operator(j):
    if (Tokens[j].token_type == Token_type.EqualOp or Tokens[j].token_type == Token_type.LessThanOp or
         Tokens[j].token_type == Token_type.GreaterThanOp or Tokens[j].token_type == Token_type.greaterThanOrEqual or
           Tokens[j].token_type == Token_type.lessThanOrEqual):
        return True
    else:
        return False

## Parser

In [148]:
def Match(a,j):
    output=dict()
    if(j<len(Tokens)):
        Temp=Tokens[j].to_dict()
        if(Temp['token_type']==a):
            j+=1
            output["node"]=[Temp['Lex']]
            output["index"]=j
            return output
        else:
            output["node"]=["error"]
            output["index"]=j+1
            errors.append("Syntax error : "+Temp['Lex']+" Expected dot")
            return output
    else:
        output["node"]=["error"]
        output["index"]=j+1
        return output

In [149]:
def list_(j):
    children=[]
    my_dictio=dict()
    if (j < len(Tokens)):
            temp=Tokens[j].to_dict()
            last_index=j
            if (temp["token_type"]==Token_type.comment):
                j+=1
            if(temp["token_type"]==Token_type.left_brace):
                match_list= list(last_index)
                children.append(match_list['node'])
                match_list_ = list_(match_list['index'])
                if match_list_['node'] != None:
                    children.append(match_list_['node'])
                    last_index=match_list_['index']
                else:
                    last_index = match_list['index']
            else:
                Node=Tree("List",children)
                my_dictio['node']=None
                my_dictio['index']=j 
                return my_dictio

    else:
        last_index=j
        Node=Tree("List_4",children)
        my_dictio['node']=None
        my_dictio['index']=last_index 
        return my_dictio        

    Node=Tree("List",children)
    my_dictio['node']=Node
    my_dictio['index']=last_index 
    return my_dictio

In [150]:
def lists(j):
    children=[]
    my_dictio=dict()
    match_list= list(j)
    children.append(match_list['node'])
    match_list_ = list_(match_list['index'])
    if (match_list_['node'] != None ):
        children.append(match_list_['node'])
        Node=Tree("Lists",children)
        my_dictio['node']=Node
        my_dictio['index']=match_list_['index']
        return my_dictio
    else:   
        Node=Tree("Lists",children)
        my_dictio['node']=Node
        my_dictio['index']=match_list['index']
        return my_dictio

In [151]:
def atoms_(j):
    children=[]
    my_dictio=dict()
    if (j < len(Tokens)):
            temp=Tokens[j].to_dict()
            last_index=j
            if(temp["token_type"]==Token_type.Constant or temp["token_type"]==Token_type.string):
                match_atom= atom(last_index)
                children.append(match_atom['node'])
                last_index = match_atom['index']
                match_atom_ = list_(last_index)
                if match_atom_['node'] != None:
                    children.append(match_atom_['node'])
                    last_index=match_atom_['index']

        
            else:
                Node=Tree("Atom_2",children)
                my_dictio['node']=None
                my_dictio['index']=last_index 
                return my_dictio
    
    else:
        last_index=j
        Node=Tree("Atom_4",children)
        my_dictio['node']=None
        my_dictio['index']=last_index 
        return my_dictio        

    Node=Tree("Atom",children)
    my_dictio['node']=Node
    my_dictio['index']=last_index 
    return my_dictio

In [152]:
def atoms(j):
    children=[]
    my_dictio=dict()
    match_atom = atom(j)
    children.append(match_atom['node'])
    j = match_atom['index']
    match_atoms_ = atoms_(j)
    if match_atoms_['node'] != None:
        children.append(match_atoms_['node'])
        j=match_atoms_['index']


    Node=Tree("Atoms",children)
    my_dictio['node']=Node
    my_dictio['index']=j
    return my_dictio

In [153]:
def boolean(j):
    children=[]
    my_dictio=dict()
    if(j<len(Tokens)):
        temp=Tokens[j].to_dict()
        last_index=j
        if(temp['token_type']==Token_type.true):
            match_true = Match(Token_type.true, j)
            children.append(match_true['node'])
            last_index= match_true['index']
        elif(temp['token_type']==Token_type.false):
            match_false =Match(Token_type.false, j)
            children.append(match_false['node'])
            last_index= match_false['index']

        
    Node=Tree("Boolean",children)
    my_dictio['node']=Node
    my_dictio['index']=last_index
    return my_dictio

In [154]:
def Comparison_op(j):
    children=[]
    my_dictio=dict()
    if (Tokens[j].token_type == Token_type.EqualOp):
        match_equal=Match(Token_type.EqualOp,j)
        children.append(match_equal['node'])
    elif (Tokens[j].token_type == Token_type.greaterThanOrEqual):
        match_greaterthanorequal=Match(Token_type.greaterThanOrEqual,j)
        children.append(match_greaterthanorequal['node'])
    elif (Tokens[j].token_type == Token_type.lessThanOrEqual):
        match_lessthanorequal=Match(Token_type.lessThanOrEqual,j)
        children.append(match_lessthanorequal['node'])
    elif (Tokens[j].token_type == Token_type.GreaterThanOp):
        match_greaterthan=Match(Token_type.GreaterThanOp,j)
        children.append(match_greaterthan['node'])
    elif (Tokens[j].token_type == Token_type.LessThanOp):
        match_lessthan=Match(Token_type.LessThanOp,j)
        children.append(match_lessthan['node'])
    
    Node=Tree("Comparision OP",children)
    my_dictio['node']=Node
    my_dictio['index']=j+1
    return my_dictio

In [155]:
def identifier(j): 
    children=[]
    my_dictio=dict()
    match_id=Match(Token_type.Identifier,j)
    children.append(match_id['node'])
    
    Node=Tree("identifier",children)
    my_dictio['node']=Node
    my_dictio['index']=match_id['index']
    return my_dictio

In [156]:
def atom_identifier_(j):   
    children=[]
    my_dictio=dict()
    if (Tokens[j].token_type == Token_type.true or Tokens[j].token_type == Token_type.false
         or Tokens[j].token_type == Token_type.Constant or Tokens[j].token_type == Token_type.string):
        match_atoms = atoms(j)
        if (match_atoms['node'] != None):
            children.append(match_atoms['node']) 
            j = match_atoms['index']
        match_atom_id_ = atom_identifier_(j)
        if (match_atom_id_['node'] != None):
            children.append(match_atom_id_['node'])
            j = match_atom_id_['index']
    elif(Tokens[j].token_type == Token_type.Identifier):
        match_id = identifier(j)
        children.append(match_id['node'])
        j = match_id['index']
        if(Tokens[j].token_type == Token_type.left_brace):
            match_list = lists(j)
            children.append(match_list['node'])
            j = match_list['index']
        
        match_atom_id_2 = atom_identifier_(j)
        if (match_atom_id_2['node'] != None):
            children.append(match_atom_id_2['node'])
            j = match_atom_id_2['index']
    else:
        my_dictio['node']=None
        my_dictio['index']=j-1
        return my_dictio


    Node=Tree("Atoms_ID",children)
    my_dictio['node']=Node
    my_dictio['index']=j
    return my_dictio

In [157]:
def atoms_identifier(j):
    children=[]
    my_dictio=dict()
    if (Tokens[j].token_type == Token_type.true or Tokens[j].token_type == Token_type.false
         or Tokens[j].token_type == Token_type.Constant or Tokens[j].token_type == Token_type.string):
        match_atoms = atoms(j)
        if (match_atoms['node'] != None):
            children.append(match_atoms['node']) 
            j = match_atoms['index']
        match_atom_id_ = atom_identifier_(j)
        if (match_atom_id_['node'] != None):
            children.append(match_atom_id_['node'])
            j = match_atom_id_['index']
    elif(Tokens[j].token_type == Token_type.Identifier):
        match_id = identifier(j)
        children.append(match_id['node'])
        j = match_id['index']
        if(Tokens[j].token_type == Token_type.left_brace):
            match_list = lists(j)
            children.append(match_list['node'])
            j = match_list['index']
        match_atom_id_2 = atom_identifier_(j)
        if (match_atom_id_2['node'] != None):
            children.append(match_atom_id_2['node'])
            j = match_atom_id_2['index']
    
    else:
        my_dictio['node']=None
        my_dictio['index']=j-1
        return my_dictio
    Node=Tree("Atoms_ID",children)
    my_dictio['node']=Node
    my_dictio['index']=j
    return my_dictio

In [158]:
def constant(j): 
    children=[]
    my_dictio=dict()

    match_constant=Match(Token_type.Constant,j)
    children.append(match_constant['node'])
    Node=Tree("constant",children)
    my_dictio['node']=Node
    my_dictio['index']=match_constant['index']
    return my_dictio

In [159]:
def decf(j):
    children=[]
    my_dictio=dict()
    match_id=identifier(j)
    children.append(match_id['node'])
    if (Tokens[j].token_type == Token_type.Constant):
        match_const = constant(match_id['index'])
        children.append(match_const['node'])
    elif (Tokens[j].token_type == Token_type.Identifier):
        match_id2=identifier(match_id['index'])
        children.append(match_id2['node'])

    Node=Tree("Incf Block",children)
    my_dictio['node']=Node
    my_dictio['index']=match_id['index'] + 1
    return my_dictio

In [160]:
def id_or_constant(j): 
    children=[]
    my_dictio=dict()
    if(j<len(Tokens)):
        temp=Tokens[j].to_dict()
        last_index=j
        if(temp['token_type'] == Token_type.Identifier):
            match_id = identifier(last_index)
            children.append(match_id['node'])
            last_index= match_id['index']
        elif(temp['token_type']== Token_type.Constant):
            match_constant=constant(last_index)
            children.append(match_constant['node'])
            last_index= match_constant['index']

    Node=Tree("id or constant",children)
    my_dictio['node']=Node
    my_dictio['index']=j+1
    return my_dictio

In [161]:
def dotimes(j):
    children=[]
    my_dictio=dict()
    match_lbrace=Match(Token_type.left_brace,j)
    children.append(match_lbrace['node'])
    
    match_id=identifier(match_lbrace['index'])
    children.append(match_id['node'])
    j = match_id['index']
    if(Tokens[j].token_type == Token_type.left_brace):
        match_lists=lists(j)
        children.append(match_lists['node'])
        j = match_lists['index']
    elif(Tokens[j].token_type == Token_type.Identifier or Tokens[j].token_type == Token_type.Constant):
        match_id_const=id_or_constant(j)
        children.append(match_id_const['node'])
        j = match_id_const['index']

    match_rbrace=Match(Token_type.right_brace,j)
    children.append(match_rbrace['node'])
    match_lists = lists(match_rbrace['index'])
    children.append(match_lists['node'])

    Node=Tree("Dotimes Block",children)
    my_dictio['node']=Node
    my_dictio['index']=match_lists['index']
    return my_dictio

In [162]:
def setq(j):
    children=[]
    my_dictio=dict()
    match_id = identifier(j)
    children.append(match_id['node'])
    j = match_id['index']

    if (Tokens[j].token_type == Token_type.left_brace):
        match_lists = lists(j)
        children.append(match_lists['node'])
        j = match_lists['index']

    elif(Tokens[j].token_type == Token_type.Identifier or Tokens[j].token_type == Token_type.Constant):
        match_id_const= id_or_constant(j)
        children.append(match_id_const['node'])
        j = match_id_const['index']

    
    Node=Tree("Setq Block",children)
    my_dictio['node']=Node
    my_dictio['index']=j
    return my_dictio

In [163]:
def defconstant(j):
    children=[]
    my_dictio=dict()
    match_id = identifier(j)
    children.append(match_id['node'])
    j = match_id['index']

    if (Tokens[j].token_type == Token_type.left_brace):
        match_lists = lists(j)
        children.append(match_lists['node'])
        j = match_lists['index']

    elif(Tokens[j].token_type == Token_type.Identifier or Tokens[j].token_type == Token_type.Constant):
        match_id_const= id_or_constant(j)
        children.append(match_id_const['node'])
        j = match_id_const['index']

    
    Node=Tree("Constant Block",children)
    my_dictio['node']=Node
    my_dictio['index']=j
    return my_dictio

In [164]:
def read(j):
    children=[]
    my_dictio=dict()
    match_id=identifier(j)
    children.append(match_id['node'])

    Node=Tree("Read Block",children)
    my_dictio['node']=Node
    my_dictio['index']=match_id['index']
    return my_dictio

In [165]:
def incf(j):
    children=[]
    my_dictio=dict()
    match_id=identifier(j)
    children.append(match_id['node'])
    j = match_id['index']

    if(Tokens[j].token_type == Token_type.left_brace):
        match_lists = lists(j)
        children.append(match_lists['node'])
        j = match_lists['index']
    elif(Tokens[j].token_type == Token_type.Constant or Tokens[j].token_type == Token_type.Identifier):
        match_id_const = id_or_constant(match_id['index'])
        children.append(match_id_const['node'])
        j = match_id_const['index']
    
    
    Node=Tree("Incf Block",children)
    my_dictio['node']=Node
    my_dictio['index']=j
    return my_dictio

In [166]:
def conditional(j):
    children=[]
    my_dictio=dict()
    if(j<len(Tokens)):
        temp=Tokens[j].to_dict()
        last_index=j
        if(temp['token_type'] == Token_type.true or temp['token_type'] == Token_type.false):
            match_bool = boolean(last_index)
            children.append(match_bool['node'])
            last_index = match_bool['index']
        elif(temp['token_type'] == Token_type.Identifier or temp['token_type'] == Token_type.Constant):
            match_id = id_or_constant(last_index)
            children.append(match_id['node'])
            last_index = match_id['index']
        elif(check_comparison_operator(last_index)):
            match_comparison = Comparison_op(last_index)
            children.append(match_comparison['node'])
            match_id1 = id_or_constant(match_comparison['index'])
            children.append(match_id1['node'])
            match_id2 = id_or_constant(match_id1['index'])
            children.append(match_id2['node'])
            last_index= match_id2['index']
    
    Node=Tree("Conditional",children)
    my_dictio['node']=Node
    my_dictio['index']=last_index
    return my_dictio

In [167]:
def if_statement(j):
    children=[]
    my_dictio=dict()
    match_conditional = conditional(j)
    children.append(match_conditional['node'])
    match_lists = lists(match_conditional['index'])
    children.append(match_lists['node'])

    
    Node=Tree("IF Block",children)
    my_dictio['node']=Node
    my_dictio['index']=match_lists['index']
    return my_dictio

In [168]:
def when(j):
    children=[]
    my_dictio=dict()
    match_lbrace=Match(Token_type.left_brace,j)
    children.append(match_lbrace['node'])
    match_conditional = conditional(match_lbrace['index'])
    children.append(match_conditional['node'])
    match_rbrace=Match(Token_type.right_brace,match_conditional['index'])
    children.append(match_rbrace['node'])
    match_lists = lists(match_rbrace['index'])
    children.append(match_lists['node'])

    Node=Tree("When Block",children)
    my_dictio['node']=Node
    my_dictio['index']=match_lists['index']
    return my_dictio

In [169]:
def Arithmeticop(j):
    children=[]
    my_dictio=dict()
    if(Tokens[j].token_type == Token_type.PlusOp):
        match_plus=Match(Token_type.PlusOp,j)
        children.append(match_plus['node'])
    elif(Tokens[j].token_type == Token_type.MinusOp):
        match_minus=Match(Token_type.MinusOp,j)
        children.append(match_minus['node'])
    elif(Tokens[j].token_type == Token_type.MultiplyOp):
        match_times=Match(Token_type.MultiplyOp,j)
        children.append(match_times['node'])
    elif(Tokens[j].token_type == Token_type.DivideOp):
        match_divide=Match(Token_type.DivideOp,j)
        children.append(match_divide['node'])
    elif(Tokens[j].token_type == Token_type.rem):
        match_rem=Match(Token_type.rem,j)
        children.append(match_rem['node'])
    elif(Tokens[j].token_type == Token_type.mod):
        match_mod=Match(Token_type.mod,j)
        children.append(match_mod['node'])
    Node=Tree("Arithmetic OP",children)
    my_dictio['node']=Node
    my_dictio['index']=j+1
    return my_dictio

In [170]:
def Logicalop(j):
    children=[]
    my_dictio=dict()
    if(Tokens[j].token_type == Token_type.and_op):
        match_and=Match(Token_type.and_op,j)
        children.append(match_and['node'])
    elif(Tokens[j].token_type == Token_type.or_op):
        match_or=Match(Token_type.or_op,j)
        children.append(match_or['node'])
    elif(Tokens[j].token_type == Token_type.not_op):
        match_or=Match(Token_type.not_op,j)
        children.append(match_or['node'])
    Node=Tree("Logical OP",children)
    my_dictio['node']=Node
    my_dictio['index']=j+1
    return my_dictio

In [171]:
def operator(j):
    children=[]
    my_dictio=dict()
    if(check_arithmetic_operator(j)):
        match_arithmetic = Arithmeticop(j)
        children.append(match_arithmetic['node'])
    elif (check_comparison_operator(j)):
        match_comparison = Comparison_op(j)
        children.append(match_comparison['node'])
    elif(check_logical_operator(j)):
        match_logical = Logicalop(j)
        children.append(match_logical['node'])
    Node=Tree("Operator",children)
    my_dictio['node']=Node
    my_dictio['index']=j+1
    return my_dictio

In [172]:
def string(j):
    children=[]
    my_dictio=dict()
    match_string=Match(Token_type.string,j)
    children.append(match_string['node'])

    Node=Tree("String",children)
    my_dictio['node']=Node
    my_dictio['index']=match_string['index']
    return my_dictio

In [173]:
def Statement(j):
   children=[]
   my_dictio=dict()
   if(j<len(Tokens)):
        temp=Tokens[j].to_dict()
        last_index=j

        if(temp["token_type"]==Token_type.when):
            out=Match(Token_type.when,last_index)
            children.append(out['node'])
            last_index=out['index']
            match_when = when(last_index)
            children.append(match_when['node'])
            last_index=match_when['index']

        elif(temp["token_type"]==Token_type.dotimes):
            out=Match(Token_type.dotimes,last_index)
            children.append(out['node'])
            last_index=out['index']
            match_dotimes = dotimes(last_index)
            children.append(match_dotimes['node'])
            last_index=match_dotimes['index']

        elif(temp["token_type"]==Token_type.if_statement):
            out=Match(Token_type.if_statement,last_index)
            children.append(out['node'])
            last_index=out['index']
            match_if = if_statement(last_index)
            children.append(match_if['node'])
            last_index=match_if['index']

        elif(temp["token_type"]==Token_type.setq):
            out=Match(Token_type.setq,last_index)
            children.append(out['node'])
            last_index=out['index']
            match_setq = setq(last_index)
            children.append(match_setq['node'])
            last_index=match_setq['index']

        elif(temp["token_type"]==Token_type.defconstant):
            out=Match(Token_type.defconstant,last_index)
            children.append(out['node'])
            last_index=out['index']
            match_defconst = defconstant(last_index)
            children.append(match_defconst['node'])
            last_index=match_defconst['index']

        elif(temp["token_type"]==Token_type.read):
            out=Match(Token_type.read,last_index)
            children.append(out['node'])
            last_index=out['index']
            match_read = read(last_index)
            children.append(match_read['node'])
            last_index=match_read['index']

        elif(temp["token_type"]==Token_type.write):
            out=Match(Token_type.write,last_index)
            children.append(out['node'])
            last_index=out['index']
            match_write = write(last_index)
            children.append(match_write['node'])
            last_index=match_write['index']

        elif(temp["token_type"]==Token_type.print):
            out=Match(Token_type.print,last_index)
            children.append(out['node'])
            last_index=out['index']
            match_print = Print(last_index)
            children.append(match_print['node'])
            last_index=match_print['index']

        elif(temp["token_type"]==Token_type.incf):
            out=Match(Token_type.incf,last_index)
            children.append(out['node'])
            last_index=out['index']
            match_incf = incf(last_index)
            children.append(match_incf['node'])
            last_index=match_incf['index']

        elif(temp["token_type"]==Token_type.decf):
            out=Match(Token_type.decf,last_index)
            children.append(out['node'])
            last_index=out['index']
            match_decf = decf(last_index)
            children.append(match_decf['node'])
            last_index=match_decf['index']

        elif(check_arithmetic_operator(last_index) or check_comparison_operator(last_index) or check_logical_operator(last_index)):
            out = operator(last_index)
            children.append(out['node'])
            last_index=out['index']
            match_state_ = statement_(last_index)
            children.append(match_state_['node'])
            last_index=match_state_['index']
            if (Tokens[last_index].token_type == Token_type.Constant or Tokens[last_index].token_type == Token_type.Identifier ):
                out = id_or_constant(last_index)
                children.append(out['node'])
                last_index = out['index']
            elif(Tokens[last_index].token_type==Token_type.left_brace):
                out = lists(last_index)
                children.append(out['node'])
                last_index = out['index']
            
       
   else:
        out =Match(Token_type.Error,j)
        children.append(out['node'])
        last_index=out['index']


   Node=Tree("Statement",children)
   my_dictio['node']=Node
   my_dictio['index']=last_index
   return my_dictio

In [174]:
def atom(j):
    children=[]
    my_dictio=dict()
    if(Tokens[j].token_type == Token_type.Constant):
        match_constant=constant(j)
        children.append(match_constant['node'])
    elif (Tokens[j].token_type == Token_type.string):
        match_string=string(j)
        children.append(match_string['node'])
    elif(Tokens[j].token_type == Token_type.true or Tokens[j].token_type == Token_type.false):
        mathc_bool=boolean(j)
        children.append(mathc_bool['node'])
    else:
        my_dictio['node']=None
        my_dictio['index']=j
        return my_dictio
    Node=Tree("Atom",children)
    my_dictio['node']=Node
    my_dictio['index']=j+1
    return my_dictio

In [175]:
def write(j):
    children=[]
    my_dictio=dict()
    if(j<len(Tokens)):
        temp=Tokens[j].to_dict()
        last_index=j
        if(temp['token_type']==Token_type.Constant or temp['token_type']==Token_type.string):
            match_atom = atom(j)
            children.append(match_atom['node'])
            last_index= match_atom['index']
        elif(temp['token_type']==Token_type.Identifier):
            match_id=identifier(j)
            children.append(match_id['node'])
            last_index= match_id['index']
        elif(temp['token_type']==Token_type.left_brace):
            match_list = lists(j)
            children.append(match_list['node'])
            last_index= match_list['index']

    Node=Tree("Write Block",children)
    my_dictio['node']=Node
    my_dictio['index']=last_index
    return my_dictio

In [176]:
def Print(j):
    children=[]
    my_dictio=dict()
    if(j<len(Tokens)):
        temp=Tokens[j].to_dict()
        last_index=j
        if(temp['token_type']==Token_type.Constant or temp['token_type']==Token_type.string):
            match_atom = atom(j)
            children.append(match_atom['node'])
            last_index= match_atom['index']
        elif(temp['token_type']==Token_type.Identifier):
            match_id=identifier(j)
            children.append(match_id['node'])
            last_index= match_id['index']
        elif(temp['token_type']==Token_type.left_brace):
            match_list = lists(j)
            children.append(match_list['node'])
            last_index= match_list['index']

    Node=Tree("Print Block",children)
    my_dictio['node']=Node
    my_dictio['index']=last_index
    return my_dictio

In [177]:
def statement_(j):
    children=[]
    my_dictio=dict()
    if(j<len(Tokens)):
        temp=Tokens[j].to_dict()
        last_index=j
        if(temp["token_type"]==Token_type.left_brace):
            match_lsit = lists(j)
            children.append(match_lsit['node'])
            last_index=match_lsit['index']
        elif(Tokens[j].token_type == Token_type.true or Tokens[j].token_type == Token_type.false
         or Tokens[j].token_type == Token_type.Constant or Tokens[j].token_type == Token_type.string
           or Tokens[j].token_type == Token_type.Identifier):
            match_atom = atoms_identifier(last_index)
            children.append(match_atom['node'])
            last_index=match_atom['index']

    Node=Tree("Statement'",children)
    my_dictio['node']=Node
    my_dictio['index']=last_index
    return my_dictio

In [178]:
def list(j):
   children=[]
   my_dictio=dict()
   match_lbrace=Match(Token_type.left_brace,j)
   children.append(match_lbrace['node'])
   match_statment = Statement(match_lbrace['index'])
   children.append(match_statment['node'])
   match_rbrace=Match(Token_type.right_brace,match_statment['index'])
   children.append(match_rbrace['node'])

   Node=Tree("List",children)
   my_dictio['node']=Node
   my_dictio['index']=match_rbrace['index']
   return my_dictio

In [179]:
def Program():
    j=0
    Children=[]

    if(Tokens[j].token_type == Token_type.left_brace):
        list_dict = lists(j)
        Children.append(list_dict["node"])
    elif(Tokens[j].token_type != Token_type.left_brace):
        atoms_dict = atoms(j)
        Children.append(atoms_dict["node"])

    
    Node=Tree('Program',Children)
    
    return Node

In [180]:
#GUI
root= tk.Tk()

canvas1 = tk.Canvas(root, width=400, height=300, relief='raised')
canvas1.pack()

label1 = tk.Label(root, text='Scanner Phase')
label1.config(font=('helvetica', 14))
canvas1.create_window(200, 25, window=label1)

label2 = tk.Label(root, text='FIle Path:')
label2.config(font=('helvetica', 10))
canvas1.create_window(200, 100, window=label2)

entry1 = tk.Entry(root) 
canvas1.create_window(200, 140, window=entry1)

def Scan():
    x1 = entry1.get()
    
    file = open(x1, 'r')
    l = file.readlines()
    find_token(l)
    df=pandas.DataFrame.from_records([t.to_dict() for t in Tokens_comment])
    # print(df)
      
    
    
    #to display token stream as table
    dTDa1 = tk.Toplevel()
    dTDa1.title('Token Stream')
    dTDaPT = pt.Table(dTDa1, dataframe=df, showtoolbar=True, showstatusbar=True)
    dTDaPT.show()

    
     
    # start Parsing
    Node=Program()
     
    
    # to display errorlist
    df1=pandas.DataFrame(errors)
    dTDa2 = tk.Toplevel()
    dTDa2.title('Error List')
    dTDaPT2 = pt.Table(dTDa2, dataframe=df1, showtoolbar=True, showstatusbar=True)
    dTDaPT2.show()
    Node.draw()
    
    
button1 = tk.Button(text='Scan', command=Scan, bg='brown', fg='white', font=('helvetica', 9, 'bold'))
canvas1.create_window(200, 180, window=button1)
root.mainloop()
